In [1]:
# relevant libraries
import PyAPD
from pykeops.torch import LazyTensor
import copy
import torch
import matplotlib.pyplot as plt

from torchmin import minimize as minimize_torch

# typical setup
device = "cuda" if torch.cuda.is_available() else "cpu"
dt = torch.float64
torch.set_default_dtype(dt)

[KeOps] Warning : Cuda libraries were not detected on the system ; using cpu only mode


In [2]:
N = 10 # number of grains
apd1 = PyAPD.apd_system(
    N = N,
    seed = 5, # fix random seed
    pixel_size_prefactor = 1.0
)
apd1.check_optimality()
#apd1.find_optimal_W()
apd1.Lloyds_algorithm(K=10)
#apd1.plot_apd()

# Save the data:
W_true = copy.deepcopy(apd1.W.contiguous())
As_true = copy.deepcopy(apd1.As.contiguous())
X_true = copy.deepcopy(apd1.X.contiguous())

# Save the grain map:
Y = copy.deepcopy(apd1.Y)
II = copy.deepcopy(apd1.assemble_apd())
II = II + 1
y = LazyTensor(Y.view(1, len(Y), 2))
print("We have ", len(Y), " pixels and ", N, " grains.")

Precision loss detected!
Percentage error =  95.62935638427734
Lloyds iteration: 0
Optimality condition successfully overwritten.
The APD is optimal!
Percentage error =  0.9365975856781006
Lloyds iteration: 1
Optimality condition successfully overwritten.
The APD is optimal!
Percentage error =  0.9365975856781006
Lloyds iteration: 2
Optimality condition successfully overwritten.
The APD is optimal!
Percentage error =  0.9365975856781006
Lloyds iteration: 3
Optimality condition successfully overwritten.
The APD is optimal!
Percentage error =  0.9365975856781006
Lloyds iteration: 4
Optimality condition successfully overwritten.
The APD is optimal!
Percentage error =  0.9365975856781006
Lloyds iteration: 5
Optimality condition successfully overwritten.
The APD is optimal!
Percentage error =  0.9365975856781006
Lloyds iteration: 6
Optimality condition successfully overwritten.
The APD is optimal!
Percentage error =  0.9365975856781006
Lloyds iteration: 7
Optimality condition successfully o

In [3]:
# rescaled objective
def objective_function_tilde(Xt,At,Wt):
    dims = At.shape
    MV = torch.einsum('bij,bj->bi', torch.index_select(At,0,II-1), Y)
    sD_ij = torch.einsum('bj,bj->b',MV,Y) - torch.index_select(Wt, 0, II-1) - torch.einsum('bi,bi->b',torch.index_select(Xt, 0, II-1),Y)
    first_sum = (-sD_ij).sum()

    at = LazyTensor(At.view(dims[0], 1, dims[1] * dims[2]))
    wt = LazyTensor(Wt.view(dims[0],1,1))
    xt = LazyTensor(Xt.view(dims[0], 1, dims[1]))
    D_ij =  (y | at.matvecmult(y)) - wt - ((y*xt).sum(dim=2))
    second_sum = (-D_ij).logsumexp(dim=0).sum(dim=0)
    
    return -(first_sum - second_sum)